In [31]:
import pandas as pd
import numpy as np
# LDA, tSNE
from sklearn.manifold import TSNE
from gensim.models.ldamodel import LdaModel
# NLTK
from nltk.tokenize import RegexpTokenizer
from nltk.stem.snowball import SnowballStemmer
from nltk.corpus import stopwords
import re
# Visualization
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import matplotlib
%matplotlib inline
import seaborn as sns
# Bokeh
from bokeh.io import output_notebook
from bokeh.plotting import figure, show
from bokeh.models import HoverTool, CustomJS, ColumnDataSource, Slider
from bokeh.layouts import column
from bokeh.palettes import all_palettes
output_notebook()

Loading BokehJS ...

In [32]:
df = pd.read_csv("papers.csv")
print(df.paper_text[0][:500])

767

SELF-ORGANIZATION OF ASSOCIATIVE DATABASE
AND ITS APPLICATIONS
Hisashi Suzuki and Suguru Arimoto
Osaka University, Toyonaka, Osaka 560, Japan
ABSTRACT
An efficient method of self-organizing associative databases is proposed together with
applications to robot eyesight systems. The proposed databases can associate any input
with some output. In the first half part of discussion, an algorithm of self-organization is
proposed. From an aspect of hardware, it produces a new style of neural netwo


In [33]:
%%time
# Removing numerals:
df['paper_text_tokens'] = df.paper_text.map(lambda x: re.sub(r'\d+', '', x))
# Lower case:
df['paper_text_tokens'] = df.paper_text_tokens.map(lambda x: x.lower())
print(df['paper_text_tokens'][0][:500])



self-organization of associative database
and its applications
hisashi suzuki and suguru arimoto
osaka university, toyonaka, osaka , japan
abstract
an efficient method of self-organizing associative databases is proposed together with
applications to robot eyesight systems. the proposed databases can associate any input
with some output. in the first half part of discussion, an algorithm of self-organization is
proposed. from an aspect of hardware, it produces a new style of neural network. in
Wall time: 3.56 s


In [34]:
%%time
df['paper_text_tokens'] = df.paper_text_tokens.map(lambda x: RegexpTokenizer(r'\w+').tokenize(x))
print(df['paper_text_tokens'][0][:25])

['self', 'organization', 'of', 'associative', 'database', 'and', 'its', 'applications', 'hisashi', 'suzuki', 'and', 'suguru', 'arimoto', 'osaka', 'university', 'toyonaka', 'osaka', 'japan', 'abstract', 'an', 'efficient', 'method', 'of', 'self', 'organizing']
Wall time: 7.15 s


In [35]:
%%time
snowball = SnowballStemmer("english")  
df['paper_text_tokens'] = df.paper_text_tokens.map(lambda x: [snowball.stem(token) for token in x])
print(df['paper_text_tokens'][0][:25])

['self', 'organ', 'of', 'associ', 'databas', 'and', 'it', 'applic', 'hisashi', 'suzuki', 'and', 'suguru', 'arimoto', 'osaka', 'univers', 'toyonaka', 'osaka', 'japan', 'abstract', 'an', 'effici', 'method', 'of', 'self', 'organ']
Wall time: 4min 2s


In [36]:
stop_en = stopwords.words('english')
df['paper_text_tokens'] = df.paper_text_tokens.map(lambda x: [t for t in x if t not in stop_en]) 
print(df['paper_text_tokens'][0][:25])

['self', 'organ', 'associ', 'databas', 'applic', 'hisashi', 'suzuki', 'suguru', 'arimoto', 'osaka', 'univers', 'toyonaka', 'osaka', 'japan', 'abstract', 'effici', 'method', 'self', 'organ', 'associ', 'databas', 'propos', 'togeth', 'applic', 'robot']


In [37]:
%%time
df['paper_text_tokens'] = df.paper_text_tokens.map(lambda x: [t for t in x if len(t) > 1])
print(df['paper_text_tokens'][0][:25])

['self', 'organ', 'associ', 'databas', 'applic', 'hisashi', 'suzuki', 'suguru', 'arimoto', 'osaka', 'univers', 'toyonaka', 'osaka', 'japan', 'abstract', 'effici', 'method', 'self', 'organ', 'associ', 'databas', 'propos', 'togeth', 'applic', 'robot']
Wall time: 1.83 s


In [38]:
df['paper_text_tokens'][0]

['self',
 'organ',
 'associ',
 'databas',
 'applic',
 'hisashi',
 'suzuki',
 'suguru',
 'arimoto',
 'osaka',
 'univers',
 'toyonaka',
 'osaka',
 'japan',
 'abstract',
 'effici',
 'method',
 'self',
 'organ',
 'associ',
 'databas',
 'propos',
 'togeth',
 'applic',
 'robot',
 'eyesight',
 'system',
 'propos',
 'databas',
 'associ',
 'ani',
 'input',
 'output',
 'first',
 'half',
 'part',
 'discuss',
 'algorithm',
 'self',
 'organ',
 'propos',
 'aspect',
 'hardwar',
 'produc',
 'new',
 'style',
 'neural',
 'network',
 'latter',
 'half',
 'part',
 'applic',
 'handwritten',
 'letter',
 'recognit',
 'autonom',
 'mobil',
 'robot',
 'system',
 'demonstr',
 'introduct',
 'let',
 'map',
 'given',
 'finit',
 'infinit',
 'set',
 'anoth',
 'finit',
 'infinit',
 'set',
 'learn',
 'machin',
 'observ',
 'ani',
 'set',
 'pair',
 'sampl',
 'random',
 'mean',
 'cartesian',
 'product',
 'comput',
 'estim',
 'make',
 'small',
 'estim',
 'error',
 'measur',
 'usual',
 'say',
 'faster',
 'decreas',
 'estim',

In [39]:
from gensim import corpora, models
np.random.seed(2017)
texts = df['paper_text_tokens'].values

In [40]:
texts

array([list(['self', 'organ', 'associ', 'databas', 'applic', 'hisashi', 'suzuki', 'suguru', 'arimoto', 'osaka', 'univers', 'toyonaka', 'osaka', 'japan', 'abstract', 'effici', 'method', 'self', 'organ', 'associ', 'databas', 'propos', 'togeth', 'applic', 'robot', 'eyesight', 'system', 'propos', 'databas', 'associ', 'ani', 'input', 'output', 'first', 'half', 'part', 'discuss', 'algorithm', 'self', 'organ', 'propos', 'aspect', 'hardwar', 'produc', 'new', 'style', 'neural', 'network', 'latter', 'half', 'part', 'applic', 'handwritten', 'letter', 'recognit', 'autonom', 'mobil', 'robot', 'system', 'demonstr', 'introduct', 'let', 'map', 'given', 'finit', 'infinit', 'set', 'anoth', 'finit', 'infinit', 'set', 'learn', 'machin', 'observ', 'ani', 'set', 'pair', 'sampl', 'random', 'mean', 'cartesian', 'product', 'comput', 'estim', 'make', 'small', 'estim', 'error', 'measur', 'usual', 'say', 'faster', 'decreas', 'estim', 'error', 'increas', 'number', 'sampl', 'better', 'learn', 'machin', 'howev', 'ex

In [ ]:
dictionary = corpora.Dictionary(texts)
corpus = [dictionary.doc2bow(text) for text in texts]
ldamodel = models.ldamodel.LdaModel(corpus, id2word=dictionary, 
                                    num_topics=8, passes=5, minimum_probability=0)

In [9]:
ldamodel.print_topics()

[(0,
  '0.020*"imag" + 0.014*"model" + 0.014*"learn" + 0.014*"use" + 0.012*"featur" + 0.010*"object" + 0.009*"train" + 0.007*"label" + 0.007*"data" + 0.006*"set"'),
 (1,
  '0.012*"network" + 0.012*"function" + 0.008*"learn" + 0.007*"system" + 0.007*"time" + 0.007*"algorithm" + 0.007*"weight" + 0.006*"comput" + 0.006*"dynam" + 0.006*"converg"'),
 (2,
  '0.030*"network" + 0.017*"train" + 0.013*"use" + 0.013*"neural" + 0.013*"layer" + 0.011*"learn" + 0.010*"input" + 0.008*"output" + 0.008*"model" + 0.008*"unit"'),
 (3,
  '0.012*"bound" + 0.010*"set" + 0.010*"estim" + 0.010*"function" + 0.010*"learn" + 0.009*"distribut" + 0.008*"algorithm" + 0.008*"sampl" + 0.007*"use" + 0.006*"data"'),
 (4,
  '0.014*"algorithm" + 0.009*"problem" + 0.009*"matrix" + 0.008*"method" + 0.008*"optim" + 0.008*"use" + 0.008*"set" + 0.007*"function" + 0.006*"comput" + 0.006*"data"'),
 (5,
  '0.027*"model" + 0.012*"use" + 0.011*"distribut" + 0.010*"data" + 0.008*"infer" + 0.008*"sampl" + 0.008*"variabl" + 0.008*"es

In [14]:
hm = np.array([[y for (x,y) in ldamodel[corpus[i]]] for i in range(len(corpus))])

In [25]:
type(corpus)

list

In [24]:
for i in ldamodel[corpus]:
    print(i)

[(0, 0.18011646), (1, 0.2017114), (2, 0.16196673), (3, 0.15094872), (4, 0.009027299), (5, 6.919227e-05), (6, 0.15030405), (7, 0.14585614)]
[(0, 9.521711e-05), (1, 0.30327225), (2, 0.02377054), (3, 9.525496e-05), (4, 9.524615e-05), (5, 9.5237374e-05), (6, 9.530971e-05), (7, 0.67248094)]
[(0, 7.0695474e-05), (1, 0.002141457), (2, 7.07449e-05), (3, 7.072719e-05), (4, 7.0750786e-05), (5, 7.0707494e-05), (6, 7.0735514e-05), (7, 0.9974342)]
[(0, 7.62844e-05), (1, 0.06187957), (2, 0.21921016), (3, 0.40853122), (4, 7.630337e-05), (5, 0.26966304), (6, 0.039152183), (7, 0.0014112357)]
[(0, 7.5504475e-05), (1, 0.03275929), (2, 0.46003687), (3, 0.50682634), (4, 7.548893e-05), (5, 7.5488264e-05), (6, 7.547887e-05), (7, 7.550606e-05)]
[(0, 0.24912885), (1, 0.0581158), (2, 0.51463), (3, 6.314225e-05), (4, 6.3154795e-05), (5, 0.17787279), (6, 6.3133135e-05), (7, 6.314181e-05)]
[(0, 0.010154747), (1, 0.1656138), (2, 0.2813125), (3, 0.12504905), (4, 0.0005116996), (5, 0.034330204), (6, 0.00012377543), (

[(0, 7.6464734e-05), (1, 0.045176767), (2, 0.019531986), (3, 0.15770926), (4, 7.646689e-05), (5, 0.0070025935), (6, 7.646575e-05), (7, 0.77035004)]
[(0, 8.0067446e-05), (1, 0.047176342), (2, 0.12298445), (3, 0.032274388), (4, 8.008218e-05), (5, 0.0006373519), (6, 0.79668725), (7, 8.0084166e-05)]
[(0, 8.314574e-05), (1, 0.18672687), (2, 0.18553172), (3, 8.316979e-05), (4, 8.315328e-05), (5, 0.5848311), (6, 0.021524955), (7, 0.021135911)]
[(0, 9.754446e-05), (1, 0.25302073), (2, 0.2607743), (3, 9.750264e-05), (4, 9.752642e-05), (5, 9.751361e-05), (6, 9.7519114e-05), (7, 0.4857174)]
[(0, 6.505666e-05), (1, 0.09044845), (2, 0.043559093), (3, 0.01225612), (4, 0.1518017), (5, 0.20472072), (6, 0.41655302), (7, 0.08059587)]
[(0, 0.016645502), (1, 0.32915986), (2, 0.031337526), (3, 8.185419e-05), (4, 8.183805e-05), (5, 8.18229e-05), (6, 8.181729e-05), (7, 0.6225298)]
[(0, 6.4437285e-05), (1, 0.1114186), (2, 0.6708476), (3, 6.442324e-05), (4, 6.442209e-05), (5, 0.21741168), (6, 6.443088e-05), (7

[(0, 7.578516e-05), (1, 0.6766029), (2, 0.22146161), (3, 0.0654888), (4, 7.579996e-05), (5, 7.5810916e-05), (6, 7.5769494e-05), (7, 0.036143474)]
[(0, 0.013802192), (1, 0.07673253), (2, 7.2136296e-05), (3, 7.214207e-05), (4, 7.215402e-05), (5, 0.0024204853), (6, 7.212139e-05), (7, 0.9067562)]
[(0, 0.00010327655), (1, 0.14612406), (2, 0.1653794), (3, 0.017504174), (4, 0.00021904678), (5, 0.2800058), (6, 0.36387703), (7, 0.026787218)]
[(0, 0.051617645), (1, 0.27153164), (2, 0.21496059), (3, 0.032450493), (4, 0.06592122), (5, 0.1384602), (6, 0.15419558), (7, 0.070862584)]
[(0, 9.177508e-05), (1, 0.6435207), (2, 0.018957602), (3, 0.10406533), (4, 0.20391141), (5, 0.029269664), (6, 9.176866e-05), (7, 9.177466e-05)]
[(0, 0.0021097332), (1, 0.4212784), (2, 9.360631e-05), (3, 0.019341595), (4, 9.359905e-05), (5, 0.014828901), (6, 9.357314e-05), (7, 0.54216063)]
[(0, 8.915323e-05), (1, 0.4248988), (2, 0.35023528), (3, 0.13879862), (4, 0.08571071), (5, 8.9140136e-05), (6, 8.9152905e-05), (7, 8.9

[(0, 0.26157948), (1, 5.4291333e-05), (2, 0.5910472), (3, 5.429505e-05), (4, 5.42806e-05), (5, 5.429061e-05), (6, 0.01392395), (7, 0.1332322)]
[(0, 0.015827175), (1, 0.033549197), (2, 0.46195245), (3, 0.04650892), (4, 0.000647794), (5, 0.44134855), (6, 8.4192245e-05), (7, 8.168944e-05)]
[(0, 7.216767e-05), (1, 0.3014463), (2, 7.2154624e-05), (3, 7.218702e-05), (4, 7.215979e-05), (5, 7.215441e-05), (6, 7.214707e-05), (7, 0.6981207)]
[(0, 0.022187103), (1, 0.006889433), (2, 0.09860923), (3, 0.04808038), (4, 0.004594362), (5, 0.81949985), (6, 6.982927e-05), (7, 6.9815236e-05)]
[(0, 7.576014e-05), (1, 0.011669061), (2, 0.018154085), (3, 7.5756965e-05), (4, 7.576027e-05), (5, 7.575131e-05), (6, 0.37900752), (7, 0.59086627)]
[(0, 3.9521947e-05), (1, 0.043275397), (2, 0.061915983), (3, 3.95221e-05), (4, 3.9525148e-05), (5, 3.9521565e-05), (6, 0.5803995), (7, 0.314251)]
[(0, 9.790334e-05), (1, 0.18507849), (2, 9.7940465e-05), (3, 0.09633265), (4, 0.0016146011), (5, 9.795169e-05), (6, 0.7165825

[(0, 0.09521513), (1, 0.20893113), (2, 0.022457726), (3, 0.00019217198), (4, 9.645492e-05), (5, 0.41227597), (6, 0.1633349), (7, 0.09749652)]
[(0, 7.448287e-05), (1, 0.5392896), (2, 7.445279e-05), (3, 7.4461626e-05), (4, 0.38942426), (5, 0.05308621), (6, 0.017902022), (7, 7.4484495e-05)]
[(0, 0.20149939), (1, 0.15759833), (2, 7.1338414e-05), (3, 0.21353856), (4, 0.0046067415), (5, 0.13407576), (6, 0.22964013), (7, 0.058969736)]
[(0, 9.2322254e-05), (1, 9.233463e-05), (2, 0.09619988), (3, 9.232361e-05), (4, 0.32425973), (5, 0.1193181), (6, 9.230039e-05), (7, 0.45985302)]
[(0, 7.579608e-05), (1, 0.2540196), (2, 0.21803817), (3, 0.52756315), (4, 7.583721e-05), (5, 7.582136e-05), (6, 7.580993e-05), (7, 7.581321e-05)]
[(0, 0.3528755), (1, 0.07322452), (2, 7.99793e-05), (3, 7.999433e-05), (4, 0.031106135), (5, 0.23467772), (6, 7.9978905e-05), (7, 0.3078762)]
[(0, 7.2729345e-05), (1, 0.16985497), (2, 0.10318617), (3, 0.49007052), (4, 0.2365974), (5, 7.273421e-05), (6, 7.274876e-05), (7, 7.273

...

[(0, 4.740355e-05), (1, 0.017016176), (2, 4.7414702e-05), (3, 4.7427246e-05), (4, 0.9826993), (5, 4.740914e-05), (6, 4.7455498e-05), (7, 4.7400186e-05)]
[(0, 0.38425118), (1, 0.007231799), (2, 0.46439156), (3, 0.016759256), (4, 6.445159e-05), (5, 0.0026356454), (6, 0.0142930895), (7, 0.11037304)]
[(0, 0.09630225), (1, 4.9652754e-05), (2, 4.9660644e-05), (3, 0.08648261), (4, 0.24256727), (5, 0.0034892096), (6, 0.57100964), (7, 4.9659793e-05)]
[(0, 0.0029657308), (1, 3.5290494e-05), (2, 0.7388606), (3, 0.0029253166), (4, 0.25510713), (5, 3.529189e-05), (6, 3.5285397e-05), (7, 3.52905e-05)]
[(0, 5.0943894e-05), (1, 0.0071575237), (2, 5.0947267e-05), (3, 5.0945564e-05), (4, 0.9925368), (5, 5.094501e-05), (6, 5.093521e-05), (7, 5.0950293e-05)]
[(0, 0.10582286), (1, 4.7813577e-05), (2, 0.25422785), (3, 0.617251), (4, 4.7823654e-05), (5, 4.7818037e-05), (6, 0.017149871), (7, 0.0054049883)]
[(0, 5.0491457e-05), (1, 0.001311412), (2, 5.0512528e-05), (3, 0.24678469), (4, 0.60839874), (5, 0.11628

[(0, 4.3969707e-05), (1, 4.3966444e-05), (2, 4.3962486e-05), (3, 0.01974953), (4, 0.041438296), (5, 0.055984568), (6, 0.87205535), (7, 0.010640341)]
[(0, 0.000101033096), (1, 0.32183444), (2, 0.67755944), (3, 0.00010099165), (4, 0.000101018355), (5, 0.000101000034), (6, 0.00010100899), (7, 0.00010103449)]
[(0, 0.03198876), (1, 4.1674655e-05), (2, 0.015253831), (3, 0.7108231), (4, 0.010181088), (5, 0.21438704), (6, 4.1671283e-05), (7, 0.017282853)]
[(0, 0.1125619), (1, 4.3250737e-05), (2, 0.86894256), (3, 0.0021408568), (4, 0.014736637), (5, 4.3266627e-05), (6, 0.0014882761), (7, 4.3262804e-05)]
[(0, 0.3560033), (1, 3.962797e-05), (2, 0.47700313), (3, 3.963078e-05), (4, 3.963165e-05), (5, 0.1375402), (6, 3.9638402e-05), (7, 0.029294867)]
[(0, 3.6628e-05), (1, 0.0031848568), (2, 3.664365e-05), (3, 0.4245605), (4, 0.23370023), (5, 3.662827e-05), (6, 0.33840787), (7, 3.662221e-05)]
[(0, 0.010687298), (1, 3.886543e-05), (2, 3.886467e-05), (3, 0.15300278), (4, 0.8230059), (5, 3.8865655e-05),

[(0, 0.00010217886), (1, 0.46709043), (2, 0.35840693), (3, 0.08942807), (4, 0.07627334), (5, 0.00010219815), (6, 0.008494626), (7, 0.00010218792)]
[(0, 0.118915714), (1, 3.6848032e-05), (2, 0.0016091573), (3, 0.00015637289), (4, 0.06267075), (5, 0.332983), (6, 3.6841844e-05), (7, 0.48359135)]
[(0, 0.019848842), (1, 0.015124785), (2, 0.00068610336), (3, 0.076678835), (4, 0.7382814), (5, 8.155414e-05), (6, 0.1492565), (7, 4.1914944e-05)]
[(0, 0.015632022), (1, 0.003071957), (2, 0.021268168), (3, 0.0005856651), (4, 0.8673272), (5, 0.053426277), (6, 0.03865363), (7, 3.5070014e-05)]
[(0, 0.30373237), (1, 4.8871174e-05), (2, 0.006315085), (3, 0.0042308704), (4, 0.6565989), (5, 0.02817143), (6, 4.8862094e-05), (7, 0.00085363706)]
[(0, 4.356489e-05), (1, 4.3550317e-05), (2, 4.3554668e-05), (3, 0.40392935), (4, 0.38085786), (5, 0.20216824), (6, 4.3562504e-05), (7, 0.012870328)]
[(0, 3.8765094e-05), (1, 0.28197786), (2, 0.004230181), (3, 0.00042264318), (4, 0.104140624), (5, 0.60123885), (6, 0.0

[(0, 8.09297e-05), (1, 0.26904967), (2, 0.25276324), (3, 0.4752905), (4, 8.0973754e-05), (5, 0.0025728154), (6, 8.093563e-05), (7, 8.096437e-05)]
[(0, 5.19493e-05), (1, 5.1956315e-05), (2, 0.761364), (3, 5.1952382e-05), (4, 0.047949158), (5, 5.195485e-05), (6, 5.1940777e-05), (7, 0.1904271)]
[(0, 0.07345294), (1, 0.0077490686), (2, 0.38735655), (3, 0.029843729), (4, 0.05297587), (5, 0.012127628), (6, 0.06718698), (7, 0.36930722)]
[(0, 0.4454968), (1, 0.15792686), (2, 0.310169), (3, 0.006173851), (4, 0.035173576), (5, 0.0020245244), (6, 8.660362e-05), (7, 0.04294883)]
[(0, 7.779195e-05), (1, 0.69870096), (2, 0.043386657), (3, 0.24042715), (4, 0.016824635), (5, 7.780542e-05), (6, 0.00042723506), (7, 7.78074e-05)]
[(0, 6.0857954e-05), (1, 0.34213585), (2, 0.13219625), (3, 0.52377814), (4, 0.001646247), (5, 6.0872197e-05), (6, 6.086985e-05), (7, 6.086719e-05)]
[(0, 5.9864688e-05), (1, 0.28121355), (2, 0.37363198), (3, 5.9861053e-05), (4, 5.986977e-05), (5, 5.9884846e-05), (6, 5.987052e-05)

[(0, 0.000108234104), (1, 0.06842233), (2, 0.19345209), (3, 0.023236299), (4, 0.17683882), (5, 0.00010822005), (6, 0.023864742), (7, 0.5139693)]
[(0, 7.661685e-05), (1, 0.022883477), (2, 0.17113578), (3, 7.661831e-05), (4, 7.6614604e-05), (5, 7.660524e-05), (6, 0.03989704), (7, 0.76577723)]
[(0, 8.8782785e-05), (1, 8.8768975e-05), (2, 0.7083617), (3, 0.006008075), (4, 8.8768575e-05), (5, 0.2831861), (6, 0.0020890848), (7, 8.874234e-05)]
[(0, 9.16536e-05), (1, 0.014594879), (2, 9.164785e-05), (3, 0.006407271), (4, 9.16537e-05), (5, 9.166157e-05), (6, 9.165001e-05), (7, 0.9785396)]
[(0, 8.105832e-05), (1, 0.025770169), (2, 0.6505792), (3, 0.1684883), (4, 8.103062e-05), (5, 8.105442e-05), (6, 0.15483813), (7, 8.1049366e-05)]
[(0, 8.787697e-05), (1, 0.40702236), (2, 0.11856856), (3, 0.4003619), (4, 0.023611797), (5, 0.05017172), (6, 8.789009e-05), (7, 8.7884306e-05)]
[(0, 0.066902354), (1, 6.2903935e-05), (2, 0.15824144), (3, 6.29095e-05), (4, 6.28945e-05), (5, 6.288498e-05), (6, 0.7257266

In [21]:
hm

array([[1.8024054e-01, 2.0195912e-01, 1.6186452e-01, ..., 6.9197820e-05,
        1.5040314e-01, 1.4580582e-01],
       [9.5217023e-05, 3.0325204e-01, 2.3804639e-02, ..., 9.5237287e-05,
        9.5309624e-05, 6.7246705e-01],
       [7.0695271e-05, 2.1769723e-03, 7.0744703e-05, ..., 7.0707269e-05,
        7.0735310e-05, 9.9739867e-01],
       ...,
       [3.2596018e-02, 8.0018413e-01, 1.2735254e-01, ..., 1.0193521e-04,
        1.0191848e-04, 3.9459590e-02],
       [1.3009518e-01, 1.1511738e-02, 2.2198096e-02, ..., 7.2863557e-05,
        7.1651936e-03, 8.2881117e-01],
       [6.1209077e-01, 6.3899482e-05, 9.6457802e-02, ..., 6.3892388e-05,
        2.7368774e-03, 2.8845900e-01]], dtype=float32)

In [15]:
tsne = TSNE(random_state=2017, perplexity=30, early_exaggeration=120)
embedding = tsne.fit_transform(hm)
embedding = pd.DataFrame(embedding, columns=['x','y'])
embedding['hue'] = hm.argmax(axis=1)

In [20]:
source = ColumnDataSource(
        data=dict(
            x = embedding.x,
            y = embedding.y,
            colors = [all_palettes['Set1'][8][i] for i in embedding.hue],
            title = df.title,
            year = df.year,
            alpha = [0.9] * embedding.shape[0],
            size = [7] * embedding.shape[0]
        )
    )
hover_tsne = HoverTool(names=["df"], tooltips="""
    <div style="margin: 10">
        <div style="margin: 0 auto; width:300px;">
            <span style="font-size: 12px; font-weight: bold;">Title:</span>
            <span style="font-size: 12px">@title</span>
            <span style="font-size: 12px; font-weight: bold;">Year:</span>
            <span style="font-size: 12px">@year</span>
        </div>
    </div>
    """)
tools_tsne = [hover_tsne, 'pan', 'wheel_zoom', 'reset']
plot_tsne = figure(plot_width=700, plot_height=700, tools=tools_tsne, title='Papers')
plot_tsne.circle('x', 'y', size='size', fill_color='colors', 
                 alpha='alpha', line_alpha=0, line_width=0.01, source=source, name="df")

callback = CustomJS(args=dict(source=source), code=
    """
    var data = source.data;
    var f = cb_obj.value
    x = data['x']
    y = data['y']
    colors = data['colors']
    alpha = data['alpha']
    title = data['title']
    year = data['year']
    size = data['size']
    for (i = 0; i < x.length; i++) {
        if (year[i] <= f) {
            alpha[i] = 0.9
            size[i] = 7
        } else {
            alpha[i] = 0.05
            size[i] = 4
        }
    }
    source.change.emit();
    """)

slider = Slider(start=df.year.min(), end=df.year.max(), value=2016, step=1, title="Before year")
slider.js_on_change('value', callback)

layout = column(slider, plot_tsne)
show(layout)